In [2]:
import sqlite3
import json
import os

# Dictionary of top players
top_players = [
    {'name': 'Magnus_Carlsen', 'fide_number': '1503014', 'chess_com_username': 'magnuscarlsen'},
    {'name': 'Fabiano_Caruana', 'fide_number': '2020009', 'chess_com_username': 'fabianocaruana'},
    {'name': 'Hikaru_Nakamura', 'fide_number': '2016192', 'chess_com_username': 'hikaru'},
    {'name': 'Arjun_Erigaisi', 'fide_number': '35009192', 'chess_com_username': 'ghandeevam2003'},
    {'name': 'Gukesh_Dommaraju', 'fide_number': '46616543', 'chess_com_username': 'gukeshdommaraju'},
    {'name': 'Nodirbek_Abdusattorov', 'fide_number': '14204118', 'chess_com_username': 'chesswarrior7197'},
    {'name': 'Alireza-Firouzja', 'fide_number': '12573981', 'chess_com_username': 'firouzja2003'},
    {'name': 'Ian_Nepomniachtchi', 'fide_number': '4168119', 'chess_com_username': 'lachesisq'},
    {'name': 'Yi_Wei', 'fide_number': '8603405', 'chess_com_username': 'wei-yi'},
    {'name': 'Viswanathan_Anand', 'fide_number': '5000017', 'chess_com_username': 'thevish'}
]

# File directories
chess_com_dir = "../data/Chess_com"
fide_dir = "../data/Fide"
database_path = "../data/database/chess_players.db"

# Ensure the database directory exists
os.makedirs(os.path.dirname(database_path), exist_ok=True)

# Initialize SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS ChessCom (
    name TEXT,
    username TEXT,
    key TEXT,
    value TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Fide (
    name TEXT,
    fide_number TEXT,
    key TEXT,
    value TEXT
)
""")

# Function to explode JSON and insert data into a table
def explode_and_insert(table, name, identifier, identifier_value, data):
    for key, value in data.items():
        cursor.execute(f"INSERT INTO {table} (name, {identifier}, key, value) VALUES (?, ?, ?, ?)", (name, identifier_value, key, json.dumps(value)))

# Populate ChessCom table
for player in top_players:
    name = player['name']
    username = player['chess_com_username']
    file_path = os.path.join(chess_com_dir, f"{name}_stats.json")

    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
            explode_and_insert("ChessCom", name, "username", username, data)

# Populate Fide table
for player in top_players:
    name = player['name']
    fide_number = player['fide_number']
    file_path = os.path.join(fide_dir, f"{name}_fide.json")

    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            data = json.load(file)
            explode_and_insert("Fide", name, "fide_number", fide_number, data)

# Commit changes and close the connection
conn.commit()
conn.close()

print(f"Data successfully combined into {database_path}")


Data successfully combined into ../data/database/chess_players.db
